# `QFit` Advanced Tips

Tianpu Zhao, Danyang Chen and Jens Koch

This guide will walk you through some important tips and usages for you to make full use of `qfit`. 

Note: This guide is still under developement and it will be updated as we make progress in `qfit` main dev.

## 1. Data file format, the importer

#### 1.1 Requirements for the data files
`QFit` supports loading multiple spectroscopy data files and fit against all of them. There are a few requirements for the data files being loaded together:
1. Each data file represent a single run two-tone spectroscopy experiment. It should  contains 
    - a few 2D spectroscopy arrays with the same shape (usually amplitude and phase) 
    - a few 1D arrays for coordinates. They should have the same length as the spectroscopy arrays' x or y dimension. When they are absent, especially when the data is just an image, `QFit` will generate the coordinate automatically.
2. When being fit together, the data files should contain the same coordinates' names. 

With the above two requirements, `QFit` accepts various input data formats including hdf5, matlab, and images. We hope to make `qfit` as easy to use as possible. Reach out to us if you want to use `qfit` with your own data format.

#### 1.2 Importing data files
You are given two piece of spectroscopy data on a fluxonium qubit. One scan contains a wide range of frequencies and flux biases, while the other scan is a zoom-in one around the qubit's sweet spot. Your goal is to fit the two scans together to extract the qubit's parameters.

In [ ]:
# Quantum model for a single fluxonium
import scqubits as scq
fluxonium = scq.Fluxonium(
    EJ = 3.0,
    EC = 0.9,
    EL = 0.25,
    flux = 0.5,
    cutoff = 100,
    truncated_dim = 5,
    id_str = "Fluxonium"
)
hilbert_space = scq.HilbertSpace([fluxonium])

# spectroscopy data
data = [
    './example_data/joint_qubit_twotone.h5',       # the broad scan
    './example_data/trans_twotone.h5'              # the zoom-in scan
]

# launch QFit. 
from qfit import Fit
fit = Fit(hilbert_space, data)

After launching `QFit`, two images are already loaded in the file tabs, where you can switch between files, add and remove files.
<p align="center">
  <img width="400" src="resources/images/tip_file_tabs.png">
</p>  

#### 1.3 Configuring the data
Although `QFit` automatically detects the compatible x and y coordinates, as shown in the <span style="color:rgb(190, 130, 250);">METADATA</span> panel, you need to tell `QFit` what are ones that are actually used during the fitting. In this case, we specify "voltage" as the only x coordinate and "frequency" as the y coordinate. 
<p align="center">
  <img width="300" src="resources/images/example_axes_select.png">
</p> 

There are a few things to note:
- Only common coordinates in all files are shown in the selection panel
- Once selected, the data may be transposed to match the selected coordinates. 
- Multiple x coordinates are allowed, while there should be only one y coordinate (frequency)

## 2. Calibrate flux crosstalk

To be completed soon...

## 3. Setting panel
You can find the setting button on the bottom left corner of the main window. Once clicked, a panel will pop up with multiple tabs: visual, spectrum and fit.

#### 3.1 Visual
The visual settings may improves the image contrast and help you locate peaks easier. 
<p align="center">
    <img width="200" src="resources/images/tip_settings_visual.png">
</p>

#### 3.2 Spectrum
Options in the spectrum settings can change what transitions are plotted, and how these transitions are marked.

<p align="center">
    <img width="200" src="resources/images/tip_settings_spectrum.png">
</p>

1. TRANSITIONS: transitions that **only** excites the selected subsystem selected are labelled in color; the dashed lines are other transitions. 
2. INITIAL STATE: the initial state of the transition, can either be specified in bare state label or dressed state index.
3. EVALS COUNT: number of eigenvalues evaluated during parameter sweep.
4. POINTS ADDED: to make the transition curve look smooth, we include more sweep points for parameter sweep, in addition to those included in the extracted data.
5. PHOTONS: number of photons involved in the transition process. The transition frequency is ${|\mathrm{final\,energy} - \mathrm{initial\,energy}|}/{\mathrm{photon\,number}}$.


**Examples**:

* If you want to see the frequencies of transitions that start from the state labelled by (qubit, resonator) = (1,0) (a very good reason for this is that for fluxonium at half-flux sweet spot, the occupation probability of the state 1 may be significantly large due to small 01 transition frequency), then you may specify (1,0) as the initial state.

<p align="center">
  <img width="600" src="resources/images/initial_state_10.png">
</p>  

* If you want to see frequencies of transitions that involves two photons, then you may specify photons to be 2.

<p align="center">
  <img width="600" src="resources/images/two_photon_process.png">
</p>  

* To see sideband transitions (transitions that involve changes in bare state label of more than one subsystems, such as (1,0)->(0,1)), you may select "none selected" for the transitions option. This option highlights and labels all transitions between states with identified bare labels.

<p align="center">
  <img width="600" src="resources/images/all_transitions.png">
</p>  

#### 3.3 Fit
`QFit` uses `scipy.optimize` module to fit the data. 
Before the fitting process, you can set the optimizer configurations in the fit tab, including the optimizer to use and the tolerance of the optimizer. For more information, check out the [scipy.optimize documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html).
<p align="center">
    <img width="200" src="resources/images/tip_settings_fit.png">
</p>

## Understanding the status

To be completed soon...

## Overall tips (not limited to qfit)

#### Extraction 
1. **Choose your points wisely!** More points with distinctive x-value being selected = longer calculation time. You may want to select representative points for fitting.

2. For transitions, selecting bare or dressed labels means that you mark your initial and final states using bare indices (states are labelled by excitations of individual subsystems) or dressed indices (states are regarded as eigenstates of the entire system and labelled as 0, 1, 2, 3,... in ascending order of energy). 

3. If you do not know what label should be given to a group of points, you may choose the "Unknown" label, and come back to label it later once you have a better idea. 

4. Transitions may not necessarily start from the ground states; sometimes a higher-lying states are populated so the transitions from this high-lying state is activated.
